In [4]:
from elasticsearch import Elasticsearch
import ujson as json

## Create elastisearch instance

In [5]:
es=Elasticsearch(HOST='http://localhost',port='9200')
es = Elasticsearch()

In [7]:
#load json  as Document into elastisearch
# create an index, news_review of the loaded documnet

with open("News_Category_Dataset_v2.ndjson") as f:
    for i,line in enumerate(f):
        if i > 1000:
            break
        onres = es.index(index="news_review", body=line)
f.close()

C:\Users\Lovevick\AppData\Local\Programs\Python\Python39\lib\site-packages\elasticsearch\connection\base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.14/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [8]:
# Deleting index
# es.indices.delete(index="news_review")

index = "news_review"

# check if the created index has successfully been created
es.indices.exists(index=index)


{'acknowledged': True}

In [ ]:
# Getting posts published within a given time range
body = {
    "query":{
        "range":{
            "date":{
                "gte":"2017-07-02",
                "lte":"2017-10-02"
            }
        }
    }
}
res = es.search(index=index, body=body)
print(res)

In [ ]:
# column name is the name of the specific column present in the document
# category name signifies the specific category searched by a user

column_name = "category"
category_name = "ENTERTAINMENT"
body = {
    "query":{
        "match":{
            column_name:category_name
        }
    }
}
res = es.search(index=index, body=body)

for i in range(len(res["hits"]["hits"])):
    print(res["hits"]["hits"][i]["_source"])
    # print(len(res))


In [ ]:
# Specify the body parameter values.
# This code is an illustration of query and search technique
# The query specifies that the response should be matched with the category field of the document and return the headline field
column_name = ["category","headline"]
category_name = "CRIME"
aggregation_name = "crime_scenes"

body = {
    "query":{
        "match":{
            column_name[0]:category_name
        }
    },
    "aggregations":{
        aggregation_name:{
            "significant_text":{
                "field":column_name[1]
            }
        }
    }
}
res = es.search(index=index, body=body)
res["hits"]["hits"][0]["_source"]

In [ ]:
# This is a multi-phrase operation.
# It finds the relevant search of the specified column according to the corollary of the specified search query
column_name = "headline"
search_phrase = "Mass shootings"
body = {
    "query":{
        "match_phrase":{
            column_name:{
                "query":search_phrase
            }
        }
    }
}
res = es.search(index=index, body=body)

for i in range(len(res["hits"]["hits"])):
    print(res["hits"]["hits"][i]["_source"])

In [ ]:
# performing a multi-search query
# We need to find relevant searches with the phrase Nicky Jam, a musician
# The fields to be searched are an array representation
# per-field boosting signified by the caret symbol ^ illustrates that the given column should be of high relevance


search_phrase = "Nicky Jam"
search_fields = ["headline^2","short_description"]
body = {
    "query":{
        "multi_match":{
            "query":search_phrase,
            "fields":search_fields
        }
    }
}
res = es.search(index=index, body=body)
print(res)